In [7]:
import os
os.environ["ALPHAVANTAGE_API_KEY"] = "BVGUKZR1MHVS0T6B"



In [8]:
import os
import requests
import pandas as pd
import yfinance as yf
import numpy as np
from datasets import Dataset

# ------------------------------
# Helper: Convert Pandas → Hugging Face Dataset
# ------------------------------
def to_hf(df, schema=None):
    """Convert a pandas DataFrame to a Hugging Face Dataset. Handles empty gracefully."""
    if df is None or getattr(df, "empty", True):
        if schema:
            return Dataset.from_dict({c: [] for c in schema})
        return Dataset.from_dict({})
    if schema:
        df = df[[c for c in schema if c in df.columns]].copy()
    return Dataset.from_pandas(df.reset_index(drop=True), preserve_index=False)

# ------------------------------
# Earnings & Financials (Yahoo Finance)
# ------------------------------
def fetch_earnings(symbol: str) -> pd.DataFrame:
    """Fetch quarterly earnings dates with EPS actual vs. estimate."""
    ticker = yf.Ticker(symbol)
    try:
        df = ticker.earnings_dates  # EPS actual vs estimate
        return df.reset_index().rename(columns={"Earnings Date": "date"})
    except Exception as e:
        print("Earnings fetch failed:", e)
        return pd.DataFrame(columns=["date","EPS Estimate","Reported EPS","Surprise(%)"])


def fetch_financials(symbol: str) -> dict:
    """Fetch financial statements: income, balance sheet, cashflow."""
    ticker = yf.Ticker(symbol)
    try:
        return {
            "income_stmt": ticker.financials.reset_index(),
            "balance_sheet": ticker.balance_sheet.reset_index(),
            "cashflow": ticker.cashflow.reset_index(),
        }
    except Exception as e:
        print("Financials fetch failed:", e)
        return {
            "income_stmt": pd.DataFrame(),
            "balance_sheet": pd.DataFrame(),
            "cashflow": pd.DataFrame(),
        }

# ------------------------------
# Risk Metrics
# ------------------------------
def compute_risk_metrics(df_prices: pd.DataFrame) -> dict:
    """Compute daily returns, volatility, Sharpe ratio."""
    try:
        if df_prices.empty or "close" not in df_prices:
            return {}
        df = df_prices.copy()
        df["returns"] = df["close"].pct_change()
        mean_ret = df["returns"].mean()
        vol = df["returns"].std()
        sharpe = mean_ret / vol if vol > 0 else np.nan
        return {
            "avg_daily_return": round(mean_ret, 6),
            "volatility": round(vol, 6),
            "sharpe_ratio": round(sharpe, 3),
        }
    except Exception as e:
        print("Risk metrics failed:", e)
        return {}

# ------------------------------
# Alpha Vantage Connector (for news + indicators)
# ------------------------------
class AlphaConnector:
    def __init__(self, api_key=None):
        # Pick up API key from os.environ if not passed directly
        self.api_key = api_key or os.getenv("ALPHAVANTAGE_API_KEY")
        if not self.api_key:
            raise ValueError("Alpha Vantage API key not found. Set os.environ['ALPHAVANTAGE_API_KEY'].")

        self.base_url = "https://www.alphavantage.co/query"

    def fetch_news(self, symbol):
        """Fetch company news & sentiment (Alpha Vantage)."""
        params = {
            "function": "NEWS_SENTIMENT",
            "tickers": symbol,
            "apikey": self.api_key,
        }
        r = requests.get(self.base_url, params=params)
        data = r.json()

        if "feed" not in data:
            print("No news data:", data)
            return pd.DataFrame()

        rows = []
        for item in data["feed"]:
            rows.append({
                "published_at": item.get("time_published"),
                "source": item.get("source"),
                "title": item.get("title"),
                "summary": item.get("summary"),
                "url": item.get("url"),
                "overall_sentiment": item.get("overall_sentiment_label"),
            })
        return pd.DataFrame(rows)

    def fetch_indicator(self, symbol, indicator, interval="daily", time_period=14, series_type="close"):
        """Generic technical indicator fetch (SMA, RSI, MACD)."""
        params = {
            "function": indicator,
            "symbol": symbol,
            "interval": interval,
            "time_period": time_period,
            "series_type": series_type,
            "apikey": self.api_key,
        }
        r = requests.get(self.base_url, params=params)
        data = r.json()

        key_map = {
            "SMA": "Technical Analysis: SMA",
            "RSI": "Technical Analysis: RSI",
            "MACD": "Technical Analysis: MACD",
        }
        key = key_map.get(indicator)
        if key not in data:
            print(f"{indicator} fetch failed:", data)
            return pd.DataFrame()

        df = pd.DataFrame.from_dict(data[key], orient="index")
        df.index = pd.to_datetime(df.index)
        df.reset_index(inplace=True)
        df = df.rename(columns={"index": "date"})

        # Cast numeric values
        for col in df.columns:
            if col != "date":
                df[col] = df[col].astype(float)

        return df

# ------------------------------
# Data Ingestion Manager
# ------------------------------
class DataIngestionManager:
    def __init__(self, api_key=None):
        self.alpha = AlphaConnector(api_key)

    def fetch_all(self, symbol, start=None, end=None):
        """Fetch prices (Yahoo), news (Alpha Vantage), indicators, earnings, financials, and risk metrics."""
        datasets = {}

        # Prices from Yahoo Finance
        try:
            df_prices = yf.download(symbol, start=start, end=end, progress=False)

            # Flatten MultiIndex columns if necessary
            if isinstance(df_prices.columns, pd.MultiIndex):
                df_prices.columns = [c[0].lower() for c in df_prices.columns]

            df_prices = df_prices.reset_index().rename(columns={
                "Date": "date",
                "open": "open",
                "high": "high",
                "low": "low",
                "close": "close",
                "adj close": "adj_close",
                "volume": "volume",
            })
            df_prices["date"] = df_prices["date"].astype(str)

            datasets["prices"] = to_hf(
                df_prices, schema=["date","open","high","low","close","adj_close","volume"]
            )
        except Exception as e:
            print("Yahoo Finance fetch failed:", e)
            df_prices = pd.DataFrame()
            datasets["prices"] = to_hf(df_prices, schema=["date","open","high","low","close","adj_close","volume"])

        # News from Alpha Vantage
        datasets["news"] = to_hf(
            self.alpha.fetch_news(symbol),
            schema=["published_at","source","title","summary","url","overall_sentiment"],
        )

        # Technical Indicators
        datasets["sma"] = to_hf(
            self.alpha.fetch_indicator(symbol, "SMA", time_period=20),
            schema=["date","SMA"],
        )
        datasets["rsi"] = to_hf(
            self.alpha.fetch_indicator(symbol, "RSI", time_period=14),
            schema=["date","RSI"],
        )

        # Earnings
        datasets["earnings"] = to_hf(
            fetch_earnings(symbol),
            schema=["date","EPS Estimate","Reported EPS","Surprise(%)"],
        )

        # Financials
        fin = fetch_financials(symbol)
        datasets["income_stmt"] = to_hf(fin["income_stmt"])
        datasets["balance_sheet"] = to_hf(fin["balance_sheet"])
        datasets["cashflow"] = to_hf(fin["cashflow"])

        # Risk Metrics
        risk = compute_risk_metrics(df_prices if not df_prices.empty else pd.DataFrame())
        if risk:
            datasets["risk_metrics"] = Dataset.from_dict({k: [v] for k, v in risk.items()})
        else:
            datasets["risk_metrics"] = to_hf(pd.DataFrame(), schema=["avg_daily_return","volatility","sharpe_ratio"])

        return datasets


In [9]:
from datetime import datetime, timedelta

# Instantiate (API key picked up from environment variable ALPHAVANTAGE_API_KEY)
mgr = DataIngestionManager()
symbol = "AAPL"
start = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
end   = datetime.now().strftime("%Y-%m-%d")

# Fetch all data
datasets = mgr.fetch_all(symbol, start, end)

# ---- Inspect Results ----
print("=== Prices sample ===")
print(datasets["prices"].to_pandas().head(), "\n")

print("=== News sample ===")
print(datasets["news"].to_pandas().head(), "\n")

print("=== Earnings sample ===")
print(datasets["earnings"].to_pandas().head(), "\n")

print("=== Income Statement (shape) ===")
print(datasets["income_stmt"].to_pandas().shape)

print("=== Balance Sheet (shape) ===")
print(datasets["balance_sheet"].to_pandas().shape)

print("=== Cashflow (shape) ===")
print(datasets["cashflow"].to_pandas().shape, "\n")

print("=== Risk Metrics ===")
print(datasets["risk_metrics"])


/tmp/ipython-input-2919383409.py:160: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_prices = yf.download(symbol, start=start, end=end, progress=False)


=== Prices sample ===
         date        open        high         low       close     volume
0  2025-09-15  237.000000  238.190002  235.029999  236.699997   42699500
1  2025-09-16  237.179993  241.220001  236.320007  238.149994   63421100
2  2025-09-17  238.970001  240.100006  237.729996  238.990005   46508000
3  2025-09-18  239.970001  241.199997  236.649994  237.880005   44249600
4  2025-09-19  241.229996  246.300003  240.210007  245.500000  163741300 

=== News sample ===
      published_at            source  \
0  20251015T173000  Zacks Commentary   
1  20251015T171621          Benzinga   
2  20251015T155641          Benzinga   
3  20251015T155032          Benzinga   
4  20251015T150427          Benzinga   

                                               title  \
0  Alphabet Rises 34% in 3 Months: Buy, Sell or H...   
1  Apple iPhone 17 Pro, Pro Max Ship Times Stay S...   
2  Powell-Triggered Stock Buying Outweighs Trump ...   
3  Apple Unveils New M5 Chip - Apple  ( NASDAQ:AA... 